# Step 6: VLM - 多模态扩展

## 学习目标

1. 理解 VLM 的整体架构
2. 理解 Vision Encoder、Projection、LLM 的作用
3. **动手实现** Projection Layer

## 核心问题

如何让语言模型"看懂"图片？

```
图片 → Vision Encoder → 图像特征 → Projection → "图像 Token" → LLM → 文本输出
```

---

## 1. VLM 架构

```
┌─────────────────────────────────────────────────┐
│   图片 (224×224×3)                               │
│         ↓                                       │
│   ┌─────────────────┐                           │
│   │ Vision Encoder  │  图片 → [196, 768] 特征   │
│   │   (CLIP-ViT)    │  196 = 14×14 个 patch     │
│   └─────────────────┘                           │
│         ↓                                       │
│   ┌─────────────────┐                           │
│   │   Projection    │  768 → text_dim           │
│   │   (可训练)      │  映射到文本空间            │
│   └─────────────────┘                           │
│         ↓                                       │
│   [img_1, ..., img_196, text_1, text_2, ...]   │
│         ↓                                       │
│   ┌─────────────────┐                           │
│   │      LLM        │  生成文本回复              │
│   └─────────────────┘                           │
│         ↓                                       │
│   "这是一只猫..."                                │
└─────────────────────────────────────────────────┘
```

---

## 2. 各组件的作用

### Vision Encoder

- **作用**：将图片编码为特征向量序列
- **类比**："翻译官"，把图片这门"外语"翻译成特征
- **常用**：CLIP-ViT、SigLIP
- **输出**：[num_patches, vision_dim]，如 [196, 768]

### Projection

- **作用**：将视觉特征映射到文本嵌入空间
- **为什么需要**：Vision Encoder 和 LLM 的维度不同
- **实现**：线性层或 MLP

### LLM

- **作用**：接收图像+文本 token，生成回复
- **输入**：[image_tokens, text_tokens]
- **把图像 token 当作"特殊的文本 token"

In [ ]:
import torch
import torch.nn as nn

# 演示：维度变换
vision_dim = 768  # CLIP-ViT 输出
text_dim = 512    # LLM 嵌入维度
num_patches = 196 # 14 × 14

# 模拟图像特征
image_features = torch.randn(1, num_patches, vision_dim)
print(f"Vision Encoder 输出: {image_features.shape}")

# Projection
projection = nn.Linear(vision_dim, text_dim)
image_embeds = projection(image_features)
print(f"Projection 后: {image_embeds.shape}")

# 模拟文本嵌入
text_len = 20
text_embeds = torch.randn(1, text_len, text_dim)
print(f"文本嵌入: {text_embeds.shape}")

# 拼接
combined = torch.cat([image_embeds, text_embeds], dim=1)
print(f"拼接后（LLM 输入）: {combined.shape}")

---

## 3. 为什么冻结 Vision Encoder？

1. **利用预训练知识**：CLIP 已经学会了强大的视觉表示
2. **减少训练成本**：只训练 Projection 和 LLM
3. **防止遗忘**：避免破坏预训练的视觉能力

---

## 4. 练习：实现 Projection Layer

去 `model_exercise.py`，完成 **TODO 1**

In [ ]:
# 测试你的实现
import importlib
import model_exercise
importlib.reload(model_exercise)

model_exercise.test_projection()

---

## 5. 验证清单

- [ ] 画出 VLM 的架构图
- [ ] 解释 Vision Encoder、Projection、LLM 各自的作用
- [ ] 解释为什么通常冻结 Vision Encoder
- [ ] 实现 Projection Layer

---

## 恭喜完成！

你已经完成了 LLM/VLM 训练全流程的学习：

```
Step 1: Tokenizer    ✓ 文本 → Token
Step 2: GPT Model    ✓ Transformer 架构
Step 3: Pretrain     ✓ 下一个词预测
Step 4: SFT          ✓ 指令微调
Step 5: RLHF         ✓ 人类偏好对齐
Step 6: VLM          ✓ 多模态扩展
```

现在你已经掌握了大模型训练的核心知识！